In [17]:
#librerias 
#redes neuronales
import numpy as np
np.random.seed(4)
import matplotlib.pyplot as plt
import pandas as pd

from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense, LSTM

#modelo arima
from pmdarima import auto_arima

#regresion lineal
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error


In [18]:
#importacion y arreglo de datos
df = pd.read_csv(r"C:\Users\ninic\OneDrive - Lambton College\Term 1\BDM 1113-NoSQL data Base\project\Project-Data-Analysis-Lambton\Electricity_Generation.csv", header=None, names=range(47))
table_names = ["Canada", "Newfoundland and Labrador", "Prince Edward Island", "Nova Scotia", "New Brunswick","Quebec", "Ontario","Manitoba", "Alberta", "British Columbia", "Saskatchewan", "Yukon", "Northwest Territories", "Nunavut"]
groups = df[0].isin(table_names).cumsum()
tables = {g.iloc[0,0]: g.iloc[1:-1, :20] for k,g in df.groupby(groups)} 

df2 = tables["Canada"]
df2 = df2.iloc[1:, :]
df2.drop(df2.columns[0], axis=1, inplace=True)
nuevos_nombres = {str(anio): f'Titulo_{anio}' for anio in range(2005, 2024)}
df2.columns = nuevos_nombres
df2 = df2.reset_index(drop=True)
ind_nombres = ["Hydro / Wave / Tidal", "Wind", "Biomass / Geothermal","Solar","Uranium","Coal & Coke","Natural Gas","Oil"]  
df2.rename(index=dict(enumerate(ind_nombres)), inplace=True)
dataset = df2.T
dataset.index = pd.to_datetime(dataset.index, format='%Y')
dataset=dataset["Wind"].to_frame()
print(dataset)

                Wind
2005-01-01   1453.41
2006-01-01   2529.41
2007-01-01   3683.41
2008-01-01   4715.44
2009-01-01   7031.23
2010-01-01   8354.23
2011-01-01  11622.99
2012-01-01  13716.78
2013-01-01  17544.74
2014-01-01  21484.50
2015-01-01  26692.57
2016-01-01  30624.00
2017-01-01  31197.00
2018-01-01  32814.01
2019-01-01  32333.00
2020-01-01  37454.07
2021-01-01  35996.13
2022-01-01  50816.97
2023-01-01  54408.92


C:\Users\ninic\AppData\Local\Temp\ipykernel_22328\1276114711.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2.drop(df2.columns[0], axis=1, inplace=True)


In [19]:
#funcion para calcular rsme con dataframes
def cal_RMSE(val_reales, val_predichos):
    val_reales = val_reales.values
    val_predichos = val_predichos.values

    mse = mean_squared_error(val_reales, val_predichos)

    # Calcula el RMSE
    rmse = np.sqrt(mse)

    return rmse

In [20]:
#----------------
#RED NEURONAL


def Redneuronal(datas,periodos_validacion):
    #Definicion parametros para ajustar ddatos entrada funcion
    p = periodos_validacion
    time_step = 3
    largo_setvalidacion = p + time_step
    largo_data = len(datas)
    largo_setentrenamiento = largo_data - largo_setvalidacion
    #Definicion set entrenamiento y validacion
    set_entrenamiento = datas.iloc[:largo_setentrenamiento+1]
    set_validacion = datas.iloc[largo_setentrenamiento:largo_data]

    #Normalización del set de entrenamiento
    sc = MinMaxScaler(feature_range=(0,1)) 
    set_entrenamiento_escalado = sc.fit_transform(set_entrenamiento) 

    #Definicion de parametros
    #time_step = 3
    X_train = [] 
    Y_train = []
    m = len(set_entrenamiento_escalado) 

    #sets de entrenamiento
    for i in range(time_step,m):
        X_train.append(set_entrenamiento_escalado[i-time_step:i,0])
        Y_train.append(set_entrenamiento_escalado[i,0])

    #ajustes para el modelo keras
    X_train, Y_train = np.array(X_train), np.array(Y_train)
    X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))

    #Creacion red LSTM
    dim_entrada = (X_train.shape[1],1)
    dim_salida = 1
    na = 500

    modelo = Sequential()
    modelo.add(LSTM(units=na, input_shape=dim_entrada))
    modelo.add(Dense(units=dim_salida))

    modelo.compile(optimizer='rmsprop', loss='mse')

    modelo.fit(X_train,Y_train,epochs=200,batch_size=400)

    #Validación (predicción del valor de la energia)
    x_test = set_validacion.values
    x_test = sc.transform(x_test)
    X_test = []
    for i in range(time_step,len(x_test)):
        X_test.append(x_test[i-time_step:i,0])

    X_test = np.array(X_test)
    X_test = np.reshape(X_test, (X_test.shape[0],X_test.shape[1],1))

    if len(X_test) > 0:
        prediccion = modelo.predict(X_test)
        prediccion = sc.inverse_transform(prediccion)
    else:
        print("No hay suficientes datos para generar predicciones.")

    #ajustes del largo
    prediction_length = len(prediccion)
    validation_length = len(set_validacion)
    new_length = validation_length - prediction_length

    adjusted_validation_set = set_validacion.iloc[new_length:].copy()
    index_names = adjusted_validation_set.index

    # Convert prediction to DataFrame and adjust the column name
    prediction_df = pd.DataFrame(prediccion, columns=["Wind_prediction"])
    result = pd.concat([adjusted_validation_set.reset_index(drop=True), prediction_df.reset_index(drop=True)], axis=1)
    result.reset_index(drop=True, inplace=True)
    result.index = index_names

    #sacar RSME
    val_RSME = cal_RMSE(result["Wind"], result["Wind_prediction"])

  
    return result, result.index.name, val_RSME


df_prueba , nombrecolumnas, val_RSME = Redneuronal(dataset,3)
df_prueba
print(val_RSME)


Epoch 1/200


c:\Users\ninic\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step - loss: 0.3892
Epoch 2/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step - loss: 0.2756
Epoch 3/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step - loss: 0.2036
Epoch 4/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step - loss: 0.1483
Epoch 5/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step - loss: 0.1046
Epoch 6/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step - loss: 0.0713
Epoch 7/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step - loss: 0.0479
Epoch 8/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step - loss: 0.0329
Epoch 9/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step - loss: 0.0244
Epoch 10/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step - loss: 0.0201
Epoch 11/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - loss: 0.0179
Epoch 12/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - loss: 0.0166
Epoch 13/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step - loss: 0.0157
Epoch 14/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step - loss: 0.0149
Epoch 15/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step - loss: 0.0142
Epoch 16/200
1/1 ━━━━━━━━━━━━━━━━

c:\Users\ninic\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(


In [23]:
#---------------
#Modelo Arima
def ModeloArima(datas,periodos_validacion):
    #arreglando dataset largo
    set_entrenamiento = datas.head(len(datas)-periodos_validacion)
    set_validacion = datas.tail(periodos_validacion)
    #entrenamiento modelo arima
    modelo = auto_arima(set_entrenamiento['Wind'], seasonal=False, stepwise=True, suppress_warnings=True, error_action="ignore", n_jobs=-1)

    # Hacer un pronóstico para los próximos  años 
    pronostico = modelo.predict(n_periods=periodos_validacion)
    # Los años para los cuales se realiza el pronóstico
    años =  set_validacion.index.name

    # Crear un DataFrame para mostrar los pronósticos de forma más amigable
    pronosticos_df = pd.DataFrame(data=pronostico, index=años, columns=['Pronóstico'])

    #sacar RSME
    val_RSME = cal_RMSE(set_validacion, pronostico)

    return pronosticos_df, val_RSME


pprueba,val_RSME_Arima = ModeloArima(dataset,3)
pprueba
print(val_RSME_Arima)

c:\Users\ninic\AppData\Local\Programs\Python\Python311\Lib\site-packages\pmdarima\arima\_validation.py:76: UserWarning: stepwise model cannot be fit in parallel (n_jobs=1). Falling back to stepwise parameter search.
  warnings.warn('stepwise model cannot be fit in parallel (n_jobs=%i). '


7817.934790192866


In [28]:
def RegresionLineal(datas,periodos_validacion):
    df3 = pd.DataFrame()

    df3['Año'] = datas.index.year
    X = df3['Año'].values.reshape(-1, 1) - 2000  # Variable independiente
    y = datas['Wind'].values  # Variable dependiente

    X_train_temp, X_test, y_train_temp, y_test = train_test_split(X, y, test_size=0.1, random_state=42)
    # Entrenar el modelo de regresión lineal
    model = LinearRegression()
    model.fit(X_train_temp, y_train_temp)

    #anos a predecir
    set_validacion = datas.tail(periodos_validacion)
    #Predecir el modelo para los anos dichos
    y_predict = model.predict(X[-periodos_validacion:])
    años = set_validacion.index
    pronosticos_df = pd.DataFrame(data=y_predict, index=años, columns=['Pronóstico'])
    #print(pronosticos_df)
    
    #sacar RSME
    val_RSME = cal_RMSE(set_validacion, pronosticos_df)

    return pronosticos_df , val_RSME

pprueba1, val_RSME_regresion= RegresionLineal(dataset,3)
print(pprueba1)
print(val_RSME_regresion)




              Pronóstico
2021-01-01  42448.612368
2022-01-01  45344.473125
2023-01-01  48240.333882
6045.2241604025685


In [25]:
def RegresionLinealnoaleatoria(datas,periodos_validacion):
    df3 = pd.DataFrame()

    df3['Año'] = datas.index.year
    X = df3['Año'].values.reshape(-1, 1) - 2000  # Variable independiente
    y = datas['Wind'].values  # Variable dependiente

    X_train_manual = X[:-periodos_validacion]  # Todos excepto los últimos 4 años
    y_train_manual = y[:-periodos_validacion]
    X_test_manual = X[-periodos_validacion:]   # Últimos 4 años
    y_test_manual = y[-periodos_validacion:]

    # Entrenar el modelo de regresión lineal
    model = LinearRegression()
    model.fit(X_train_manual, y_train_manual)

    #anos a predecir
    set_validacion = datas.tail(periodos_validacion)
    #Predecir el modelo para los anos dichos
    y_predict = model.predict(X[-periodos_validacion:])
    años = set_validacion.index
    pronosticos_df = pd.DataFrame(data=y_predict, index=años, columns=['Pronóstico'])
    #print(pronosticos_df)

    return pronosticos_df

pprueba1 = RegresionLinealnoaleatoria(dataset,3)
pprueba1


,Pronóstico
2021-01-01,39909.18425
2022-01-01,42521.65600
2023-01-01,45134.12775
